In [1]:
import pickle
import numpy as np
import torch
from utils import *
from tqdm.notebook import trange, tqdm
import random
import json
import seaborn as sns
from sklearn.metrics import top_k_accuracy_score, confusion_matrix
import pandas as pd
from classes_synsets import SYNS

In [2]:
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)


In [3]:
# Load Test Data

image_embeddings = torch.load("/nethome/bdevnani3/flash1/long_tail_lang/embedding_datasets/clip/LT_Dataset_test/image_embeddings.pt").cpu()
labels = torch.load("/nethome/bdevnani3/flash1/long_tail_lang/embedding_datasets/clip/LT_Dataset_test/labels.pt").cpu()

image_embeddings_val = torch.load("/nethome/bdevnani3/flash1/long_tail_lang/embedding_datasets/clip/LT_Dataset_val/image_embeddings.pt").cpu()
labels_val = torch.load("/nethome/bdevnani3/flash1/long_tail_lang/embedding_datasets/clip/LT_Dataset_val/labels.pt").cpu()

image_embeddings_train = torch.load("/nethome/bdevnani3/flash1/long_tail_lang/embedding_datasets/clip/LT_Dataset_train/image_embeddings.pt").cpu()
labels_train = torch.load("/nethome/bdevnani3/flash1/long_tail_lang/embedding_datasets/clip/LT_Dataset_train/labels.pt").cpu()


In [4]:
inds = np.array(random.choices(range(50000), k=10000))

small_ie = image_embeddings[inds]
small_labels = labels[inds]
print(small_ie.shape, small_labels.shape)

torch.Size([10000, 1024]) torch.Size([10000])


In [5]:
# Load Model

clf = pickle.load(open("results_sklearn/config_convex_comb_1/13-Sep-2022-00-50-56/final_model.sav", 'rb'))

In [6]:
from classes import CLASSES, GENERIC_PROMPT_COLLECTIONS
import torch.nn as nn
from clip import clip
import os
import torch


# Initialize CLIP models 
class TextEncoder(nn.Module):
    def __init__(self, clip_model):
        super().__init__()
        self.transformer = clip_model.transformer
        self.positional_embedding = clip_model.positional_embedding
        self.ln_final = clip_model.ln_final
        self.text_projection = clip_model.text_projection
        self.dtype = clip_model.dtype
        self.token_embedding = clip_model.token_embedding

    def forward(self, text):
        x = self.token_embedding(text).type(self.dtype)  # [batch_size, n_ctx, d_model]

        x = x + self.positional_embedding.type(self.dtype)
        x = x.permute(1, 0, 2)  # NLD -> LND
        x = self.transformer(x)
        x = x.permute(1, 0, 2)  # LND -> NLD
        x = self.ln_final(x).type(self.dtype)
        x = x[torch.arange(x.shape[0]), text.argmax(dim=-1)] @ self.text_projection

        return x

def load_clip_to_cpu(visual_backbone):
    backbone_name = visual_backbone
    url = clip._MODELS[backbone_name]
    model_path = clip._download(url, os.path.expanduser("~/.cache/clip"))

    try:
        # loading JIT archive
        model = torch.jit.load(model_path, map_location="cpu").eval()
        state_dict = None

    except RuntimeError:
        state_dict = torch.load(model_path, map_location="cpu")

    model = clip.build_model(state_dict or model.state_dict())

    return model

clip_model = load_clip_to_cpu("RN50")

visual_model = torch.nn.DataParallel(clip_model.visual).cuda()

text_model = TextEncoder(clip_model)
text_model = torch.nn.DataParallel(text_model).cuda()

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
all_labels_text = {}
text_embs = []
text_labels = []
with torch.no_grad():
    for label in tqdm(range(1000)):
        all_labels_text[label] = []

        templates = ["a photo of a {}."]
        c = np.array(CLASSES)[label]
            
        texts = clip.tokenize([template.format(c) for template in templates]) 
        texts = texts.cuda()
        zeroshot_weights = clip_model.encode_text(texts).float()
        zeroshot_weights = zeroshot_weights / zeroshot_weights.norm(
            dim=-1, keepdim=True
        )
        all_labels_text[label].append(zeroshot_weights)
        text_embs.append(zeroshot_weights)
        for i in range(len(templates)):
            text_labels.append(label)
        
text_embs = torch.cat(text_embs)

In [ ]:
text_embs.shape

In [ ]:
# Class frequencies

freqs = json.load(open("/nethome/bdevnani3/flash1/long_tail_lang/embedding_datasets/class_frequencies.json"))
freqs = {int(k):v for k,v in freqs.items()}

In [ ]:
freqs

In [ ]:
many_labels = []
many_embs = []
many_inds = []
med_labels = []
med_embs = []
med_inds = []
few_labels = []
few_embs = []
few_inds = []

for i in tqdm(range(image_embeddings.shape[0])):
    if freqs[labels[i].item()] >= 100:
        many_labels.append(labels[i])
        many_embs.append(image_embeddings[i, :])
        many_inds.append(i)
    elif freqs[labels[i].item()] >= 20:
        med_labels.append(labels[i])
        med_embs.append(image_embeddings[i, :])
        med_inds.append(i)
    else:
        few_labels.append(labels[i])
        few_embs.append(image_embeddings[i, :])
        few_inds.append(i)
        
many_embs = torch.stack(many_embs)
med_embs = torch.stack(med_embs)
few_embs = torch.stack(few_embs)

many_inds = np.array(many_inds)
med_inds = np.array(med_inds)
few_inds = np.array(few_inds)


In [ ]:
fg_buckets = [5, 10, 15, 20, 30, 40, 60, 100, 160, 10000]

fg_indices_val = {k: [] for k in fg_buckets}

seen = set()

for i in tqdm(range(image_embeddings_val.shape[0])):
    for k in fg_buckets:
        if freqs[labels_val[i].item()] <= k and i not in seen:
            seen.add(i)
            fg_indices_val[k].append(i)
print(len([i for k in fg_indices_val for i in fg_indices_val[k]]))
[len(fg_indices_val[k]) for k in fg_indices_val]

In [ ]:


fg_indices = {k: [] for k in fg_buckets}

seen = set()

for i in tqdm(range(image_embeddings.shape[0])):
    for k in fg_buckets:
        if freqs[labels[i].item()] <= k and i not in seen:
            seen.add(i)
            fg_indices[k].append(i)
print(len([i for k in fg_indices for i in fg_indices[k]]))
[len(fg_indices[k]) for k in fg_indices]

In [ ]:
def bucket_indices(l):
    seen = set()
    out = []
    for i in tqdm(range(len(l))):
        for k,kk in enumerate(fg_buckets):
            if freqs[l[i].item()] <= kk and i not in seen:
                seen.add(i)
                out.append(k)
    return out

In [ ]:
import seaborn as sns
plt.rcParams["figure.figsize"] = (10,10)

In [ ]:
def all_accs(model):
    preds = model.predict(image_embeddings)
    l = labels.numpy()
    avg_acc = mic_acc_cal(preds, l)
    many = mic_acc_cal(preds[many_inds], many_labels)
    med = mic_acc_cal(preds[med_inds], med_labels)
    few = mic_acc_cal(preds[few_inds], few_labels)
    print(f"{avg_acc:.3f}/{many:.3f}/{med:.3f}/{few:.3f}")
    return [avg_acc, many, med, few]

In [19]:
def all_accs_p(preds):
    l = labels.numpy()
    avg_acc = mic_acc_cal(preds, l)
    many = mic_acc_cal(preds[many_inds], many_labels)
    med = mic_acc_cal(preds[med_inds], med_labels)
    few = mic_acc_cal(preds[few_inds], few_labels)
    print(f"{avg_acc:.3f}/{many:.3f}/{med:.3f}/{few:.3f}")
    return [avg_acc, many, med, few]

In [20]:
def all_accs_fg(preds, ls, fg_indices):
    avg_acc = mic_acc_cal(preds, ls)
    out_accs = [avg_acc]
    for k in fg_indices:
        indices = np.array(fg_indices[k])
        acc = mic_acc_cal(preds[indices], ls[indices])
        out_accs.append(acc)
    print([f"{k:.5f}" for k in out_accs])
    return out_accs

In [21]:
conv_comb_0_8_clf = pickle.load(open("results_sklearn/config_convex_comb_1/class_balanced/13-Sep-2022-13-41-17/final_model.sav", 'rb'))

# text_only_balanced_clf = pickle.load(open("results_sklearn/config_text_only_0/29-Sep-2022-01-32-28/final_model.sav", 'rb'))
text_only_balanced_clf = pickle.load(open("results_sklearn/config_text_only_liblinear/06-Nov-2022-01-47-08/final_model.sav", 'rb'))
text_only_balanced_cupl = pickle.load(open("results_sklearn/config_text_only_clip_and_cupl_liblinear/07-Nov-2022-12-41-14/final_model.sav", 'rb'))


image_only_clf = pickle.load(open("results_sklearn/config_image_linear_probe_0/class_balanced/13-Sep-2022-01-35-07/final_model.sav", 'rb'))
image_only_clf_2 = pickle.load(open("results_sklearn/config_image_linear_probe_0/balanced/29-Sep-2022-01-38-27/final_model.sav", 'rb'))
image_only_clf_3 = pickle.load(open("results_sklearn/config_image_linear_probe_0/balanced/07-Nov-2022-18-27-46/final_model.sav", 'rb'))
image_only_clf_4 = pickle.load(open("results_sklearn/config_image_linear_probe_0/balanced/11-Nov-2022-16-23-33/final_model.sav", 'rb'))

image_only_clf_oversampled = pickle.load(open("results_sklearn/config_image_linear_probe_oversampled_auto/class_balanced/07-Nov-2022-20-13-26/final_model.sav", 'rb'))
# image_only_clf_oversampled_500 = pickle.load(open("results_sklearn/config_image_linear_probe_0/balanced/11-Nov-2022-16-23-33/final_model.sav", 'rb'))

image_only_clf_undersampled_500 = pickle.load(open("results_sklearn/config_image_linear_probe_undersampled_0-5/12-Nov-2022-12-17-52/final_model.sav", 'rb'))
image_only_clf_undersampled = pickle.load(open("results_sklearn/config_image_linear_probe_undersampled_auto/class_balanced/07-Nov-2022-20-18-24/final_model.sav", 'rb'))

image_only_clf_oversampled = pickle.load(open("results_sklearn/config_image_linear_probe_0/c_10/13-Nov-2022-20-07-26/final_model.sav", 'rb'))
# image_only_clf_oversampled = pickle.load(open("results_sklearn/config_image_linear_probe_nojitter/01-Dec-2022-22-31-07/final_model.sav", 'rb'))





### Predict mixing coefficients at eval time

In [22]:
from data_loader import dataloaders as dataloader

In [23]:
d = dataloader.load_data(
    data_root="./datasets/ImageNet/",
    dataset="ImageNet_LT",
    phase="test",
    batch_size=1000,
    sampler_dic=None,
    num_workers=10,
    type="LT_Dataset",
    prompt_set="Imagenet",
)
dataset_test = d[0]

Loading data from /nethome/bdevnani3/flash1/long_tail_lang/data/ImageNet_LT/ImageNet_LT_test.txt
Use data transformation: Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=None)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
)
***********************DATASET: test LT_Dataset
test 50000
No sampler.
Shuffle is True.


In [24]:
import torchvision.transforms as T
from torchvision.transforms import InterpolationMode

BICUBIC = InterpolationMode.BICUBIC
t1 = T.RandomResizedCrop(size=224, scale=(0.5, 1), interpolation=BICUBIC)
t2 = T.RandomHorizontalFlip(p=1)
t3 = T.RandomGrayscale(p=1)
t4 = T.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0)

In [25]:
freqs

{91: 81,
 129: 161,
 683: 77,
 830: 343,
 450: 48,
 593: 292,
 566: 1173,
 600: 169,
 86: 1280,
 559: 310,
 83: 135,
 104: 61,
 466: 152,
 335: 109,
 367: 128,
 773: 175,
 52: 189,
 3: 352,
 347: 198,
 616: 354,
 387: 231,
 658: 253,
 486: 127,
 984: 714,
 947: 284,
 571: 340,
 907: 216,
 262: 102,
 828: 229,
 684: 42,
 417: 1053,
 490: 652,
 601: 683,
 879: 134,
 309: 306,
 81: 93,
 585: 101,
 236: 29,
 31: 83,
 770: 129,
 987: 233,
 191: 244,
 395: 26,
 637: 100,
 870: 642,
 715: 194,
 112: 393,
 2: 516,
 363: 169,
 388: 171,
 97: 819,
 737: 472,
 899: 272,
 929: 534,
 844: 175,
 922: 284,
 375: 232,
 979: 98,
 23: 171,
 499: 22,
 319: 290,
 209: 43,
 228: 41,
 765: 290,
 913: 215,
 725: 454,
 291: 53,
 33: 56,
 787: 271,
 318: 611,
 783: 229,
 970: 72,
 680: 61,
 875: 459,
 464: 224,
 365: 158,
 934: 141,
 628: 140,
 631: 57,
 554: 382,
 30: 287,
 451: 137,
 956: 80,
 134: 285,
 638: 90,
 107: 133,
 449: 236,
 194: 106,
 578: 252,
 75: 58,
 744: 56,
 978: 217,
 687: 264,
 976: 625,


In [244]:
l = 0

cropped_ims = []
cropped_im_labels = []
cropped_im_ind_labels = []
mixed_clf = pickle.load(open("results_sklearn/config_convex_comb_1/13-Sep-2022-00-50-56/final_model.sav", 'rb'))
from scipy import stats

total_acc = np.zeros(1000)
for im, label, _, path in tqdm(dataset_test):
    for k in range(len(im)):
        cropped_ims = []
        cropped_im_labels = []
        if label[k].item() == l:
            print(f"Found {l}")
            curr_im = im[k]
            cropped_ims.append(curr_im)
            cropped_im_labels.append(label[k].item())
            cropped_ims.append(t1(curr_im))
            cropped_im_labels.append(label[k].item())
            cropped_ims.append(t2(curr_im))
            cropped_im_labels.append(label[k].item())
            cropped_ims.append(t3(curr_im))
            cropped_im_labels.append(label[k].item())
            cropped_ims.append(t4(curr_im))
            cropped_im_labels.append(label[k].item())
#             for j in range(10):
#                 t = t1(curr_im)
#                 cropped_ims.append(t)
#                 cropped_im_labels.append(label[k].item())
            cropped_ims = torch.stack(cropped_ims)
            with torch.no_grad():
                x = visual_model(cropped_ims.half()).float()
                x = x / x.norm(dim=-1, keepdim=True)
            final_pred = []
            best_agreement = 0
            default_pred = 0
            all_coef_preds = []
            all_coef_pred_probas = []
            for ii in range(0, 21):
                i = ii/20.0
                mixed_clf.coef_ = i*text_only_balanced_clf.coef_ + (1-i)*image_only_clf_oversampled.coef_
                mixed_clf.intercept_ = i*text_only_balanced_clf.intercept_ + (1-i)*image_only_clf_oversampled.intercept_
                probas = mixed_clf.predict_proba(x.cpu().detach().numpy())
                preds = mixed_clf.predict(x.cpu().detach().numpy())
                all_coef_preds.append(preds)
                all_coef_pred_probas.append(np.var(probas))
                if i == 0.65:
                    default_pred = preds[0]
                mode = preds[0]
                agreement = preds == mode
#                 agreement = agreement
                agreement = np.sum(agreement)
                if agreement > best_agreement:
                    final_pred = [ii]
                    best_agreement = agreement
                if agreement == best_agreement:
                    final_pred.append(ii)
                    best_agreement = agreement
            if len(final_pred) == 1:
                final_pred = all_coef_preds[final_pred[0]][0]
            else:
                pred_ii = int(np.mean(final_pred))
                final_pred = all_coef_preds[pred_ii][0]
            print(freqs[label[k].item()], default_pred, final_pred, best_agreement)
            print(CLASSES[default_pred], CLASSES[final_pred], best_agreement)
            if final_pred == label[k].item():
                total_acc[label[k].item()] +=1
                if default_pred != label[k].item():
                    print("*********")
                    print(all_coef_preds)
            elif default_pred == label[k].item():
                print(all_coef_preds)
            if default_pred != label[k].item():
                print("xxxxxxxx")
                print(all_coef_preds)
#                 print(np.round(all_coef_pred_probas,6))
                

print(cwa[l], total_acc[l]*2)

  0%|          | 0/50 [00:00<?, ?it/s]

Found 513
75 875 875 11
trombone trombone 11
xxxxxxxx
[array([875, 875, 875, 875, 875, 875, 875, 875, 875, 875, 875]), array([875, 875, 875, 875, 875, 875, 875, 875, 875, 875, 875]), array([875, 875, 875, 875, 875, 875, 875, 875, 875, 875, 875]), array([875, 875, 875, 875, 875, 875, 875, 875, 875, 875, 875]), array([875, 875, 875, 875, 875, 875, 875, 875, 875, 875, 875]), array([875, 875, 875, 875, 875, 875, 875, 875, 875, 875, 875]), array([875, 875, 875, 875, 875, 875, 875, 875, 875, 875, 875]), array([875, 875, 875, 875, 875, 875, 875, 875, 875, 875, 875]), array([875, 875, 875, 875, 875, 875, 875, 875, 875, 875, 875]), array([875, 875, 875, 875, 875, 875, 875, 875, 875, 875, 875]), array([875, 875, 875, 875, 875, 875, 875, 875, 875, 875, 875]), array([875, 875, 875, 875, 875, 875, 875, 875, 875, 875, 875]), array([875, 875, 875, 875, 875, 875, 875, 875, 875, 875, 875]), array([875, 875, 875, 875, 875, 875, 513, 875, 875, 875, 875]), array([875, 875, 875, 875, 875, 875, 513, 875, 87

75 862 684 6
torch ocarina 6
xxxxxxxx
[array([862, 862, 875, 862, 862, 678, 699, 678, 678, 875, 678]), array([862, 862, 875, 862, 862, 678, 699, 678, 678, 875, 678]), array([862, 862, 875, 862, 862, 678, 699, 678, 678, 875, 678]), array([862, 862, 875, 862, 862, 699, 699, 699, 678, 875, 678]), array([862, 862, 875, 862, 862, 699, 699, 699, 678, 875, 678]), array([862, 862, 875, 862, 862, 699, 699, 699, 678, 875, 678]), array([862, 862, 875, 862, 862, 699, 699, 699, 678, 875, 678]), array([862, 862, 875, 862, 862, 699, 699, 699, 678, 875, 678]), array([862, 862, 875, 862, 862, 699, 699, 699, 678, 875, 678]), array([862, 862, 875, 862, 862, 699, 699, 699, 678, 875, 678]), array([862, 862, 875, 862, 862, 699, 699, 862, 678, 875, 678]), array([862, 862, 875, 862, 875, 699, 699, 862, 862, 875, 678]), array([862, 875, 875, 862, 875, 699, 699, 862, 862, 875, 678]), array([862, 875, 875, 862, 875, 699, 699, 862, 862, 875, 678]), array([862, 875, 875, 862, 875, 699, 699, 862, 862, 875, 699]), a

75 513 513 11
cornet cornet 11
Found 513
75 439 513 11
military hat (bearskin or shako) cornet 11
*********
[array([439, 875, 439, 439, 439, 439, 439, 875, 439, 875, 439]), array([439, 875, 439, 439, 439, 439, 439, 875, 439, 875, 439]), array([439, 875, 439, 439, 439, 439, 439, 875, 439, 875, 439]), array([439, 875, 439, 439, 439, 439, 439, 875, 439, 875, 439]), array([439, 875, 439, 439, 439, 439, 439, 875, 439, 875, 439]), array([439, 875, 439, 439, 439, 439, 439, 439, 439, 875, 439]), array([439, 875, 439, 439, 439, 439, 439, 439, 439, 875, 439]), array([439, 875, 439, 439, 439, 439, 439, 439, 439, 875, 439]), array([439, 875, 439, 439, 439, 439, 439, 439, 439, 875, 439]), array([439, 875, 439, 439, 439, 439, 439, 439, 439, 875, 439]), array([439, 875, 439, 439, 439, 439, 439, 439, 439, 875, 439]), array([439, 875, 439, 439, 439, 439, 439, 439, 439, 875, 439]), array([439, 875, 439, 439, 439, 439, 439, 439, 439, 875, 439]), array([439, 875, 439, 439, 439, 439, 439, 439, 439, 875, 43

75 513 875 11
cornet trombone 11
[array([875, 875, 875, 875, 875, 875, 875, 875, 875, 875, 875]), array([875, 875, 875, 875, 875, 875, 875, 875, 875, 875, 875]), array([875, 875, 875, 875, 875, 875, 875, 875, 875, 875, 875]), array([875, 875, 875, 875, 875, 875, 875, 875, 875, 875, 875]), array([875, 875, 875, 875, 875, 875, 875, 875, 875, 875, 875]), array([875, 875, 875, 875, 875, 875, 875, 875, 875, 875, 875]), array([875, 875, 875, 875, 875, 875, 875, 875, 875, 875, 875]), array([875, 875, 875, 875, 875, 875, 875, 875, 875, 875, 875]), array([875, 875, 875, 875, 875, 875, 875, 875, 875, 875, 875]), array([513, 875, 875, 875, 875, 875, 875, 875, 875, 875, 875]), array([513, 875, 875, 875, 875, 875, 513, 875, 875, 875, 875]), array([513, 875, 513, 875, 875, 875, 513, 875, 875, 875, 875]), array([513, 513, 513, 875, 875, 875, 513, 875, 875, 875, 513]), array([513, 513, 513, 875, 513, 513, 513, 875, 875, 513, 513]), array([513, 513, 513, 875, 513, 513, 513, 875, 513, 513, 513]), array(

75 439 439 11
military hat (bearskin or shako) military hat (bearskin or shako) 11
xxxxxxxx
[array([439, 439, 439, 439, 439, 439, 439, 439, 439, 439, 439]), array([439, 439, 439, 439, 439, 439, 439, 439, 439, 439, 439]), array([439, 439, 439, 439, 439, 439, 439, 439, 439, 439, 439]), array([439, 439, 439, 439, 439, 439, 439, 439, 439, 439, 439]), array([439, 439, 439, 439, 439, 439, 439, 439, 439, 439, 439]), array([439, 439, 439, 439, 439, 439, 439, 439, 439, 439, 439]), array([439, 439, 439, 439, 439, 439, 439, 439, 439, 439, 439]), array([439, 439, 439, 439, 439, 439, 439, 439, 439, 439, 439]), array([439, 439, 439, 439, 439, 439, 439, 439, 439, 439, 439]), array([439, 439, 439, 439, 439, 439, 439, 439, 439, 439, 439]), array([439, 652, 439, 652, 439, 439, 439, 439, 439, 439, 439]), array([439, 652, 439, 652, 439, 439, 439, 439, 439, 439, 439]), array([439, 652, 652, 652, 439, 439, 439, 439, 439, 439, 439]), array([439, 652, 652, 652, 439, 439, 439, 652, 439, 439, 439]), array([439,

75 513 875 9
cornet trombone 9
[array([875, 875, 875, 875, 875, 875, 566, 566, 875, 875, 875]), array([875, 875, 513, 875, 875, 875, 566, 566, 875, 875, 875]), array([875, 875, 513, 875, 875, 875, 566, 566, 875, 875, 875]), array([875, 875, 513, 875, 875, 875, 566, 566, 875, 513, 875]), array([875, 875, 513, 875, 875, 875, 566, 566, 875, 513, 875]), array([875, 875, 513, 875, 875, 875, 566, 566, 875, 513, 875]), array([875, 875, 513, 875, 875, 875, 566, 566, 875, 513, 875]), array([875, 875, 513, 875, 875, 875, 566, 566, 875, 513, 875]), array([875, 875, 513, 875, 875, 875, 566, 566, 875, 513, 875]), array([875, 875, 513, 875, 875, 875, 566, 566, 875, 513, 875]), array([875, 875, 513, 875, 875, 875, 566, 566, 875, 513, 875]), array([513, 875, 513, 875, 875, 875, 566, 566, 875, 513, 875]), array([513, 875, 513, 875, 875, 875, 566, 566, 875, 513, 875]), array([513, 875, 513, 875, 875, 875, 566, 566, 875, 513, 875]), array([513, 875, 513, 875, 875, 875, 566, 566, 875, 513, 875]), array([5

In [201]:
# Generate training text and image average prototypes

# image 

image_prototypes = np.zeros((1000, 1024))
c = np.zeros(1000)

for i,l in tqdm(enumerate(labels_train)):
    image_prototypes[l,:] =  image_prototypes[l,:] + image_embeddings_train[i,:].numpy()
    c[l] +=1
    
for i in range(1000):
    image_prototypes[i,:] /= c[i]
image_prototypes.shape

0it [00:00, ?it/s]

(1000, 1024)

In [202]:
text_embs.shape

torch.Size([1000, 1024])

In [232]:
import scipy
scipy.special.softmax([0.19264835, 0.23104969, 0.24819319, 0.25063   ])

array([0.24061828, 0.25003806, 0.25436154, 0.25498212])

In [233]:
scipy.special.softmax([0.82806292, 0.8853344 , 0.89792031, 0.9042871 ])

array([0.23750132, 0.25150042, 0.25468578, 0.25631249])

In [240]:
np.where(cwa < .50)

(array([  7,  32,  35,  36,  41,  46,  54,  55,  58,  59,  60,  62,  65,
         66,  68,  73, 110, 120, 124, 125, 151, 152, 155, 158, 163, 165,
        166, 167, 170, 171, 175, 180, 184, 187, 189, 192, 193, 196, 197,
        199, 200, 202, 204, 210, 211, 212, 217, 223, 226, 229, 231, 233,
        234, 235, 236, 237, 238, 241, 246, 248, 256, 257, 265, 266, 267,
        269, 271, 272, 277, 282, 302, 303, 312, 328, 345, 349, 356, 358,
        360, 374, 380, 381, 384, 385, 389, 409, 431, 434, 436, 438, 445,
        446, 455, 459, 461, 463, 469, 479, 482, 488, 489, 491, 493, 494,
        499, 501, 504, 505, 507, 513, 516, 523, 527, 538, 541, 542, 544,
        556, 558, 567, 572, 579, 584, 587, 590, 596, 600, 606, 615, 618,
        620, 623, 632, 633, 638, 648, 650, 651, 653, 656, 657, 664, 670,
        673, 676, 677, 680, 681, 689, 691, 696, 700, 702, 705, 708, 710,
        718, 725, 726, 728, 729, 730, 731, 733, 735, 740, 741, 742, 744,
        747, 748, 749, 750, 759, 764, 767, 771, 772

In [237]:
cwa[155]

0.0

Zero Shot K

In [248]:
import sklearn 
pairwise_cs_train = sklearn.metrics.pairwise.cosine_similarity(image_embeddings_train.cpu(), text_embs.cpu())

In [249]:
pairwise_cs_val = sklearn.metrics.pairwise.cosine_similarity(image_embeddings_val.cpu(), text_embs.cpu())

In [250]:
def top_k_acc(pairwise_cs, k, labels):
    indices = (-torch.tensor(pairwise_cs).softmax(dim=-1).numpy()).argsort(axis=-1)[:, :k]
    acc = 0
    for i in range(indices.shape[0]):
        if labels[i].item() in indices[i]:
            acc +=1.0
    acc/= indices.shape[0]
    return acc
top_k_acc(pairwise_cs_val, 1, labels_val)

0.5874

In [393]:
lams_val = np.zeros(1000)
indices = (-torch.tensor(pairwise_cs_val).softmax(dim=-1).numpy()).argsort(axis=-1)[:, :1]
for i in range(indices.shape[0]):
    if labels_val[i].item() in indices[i]:
        lams_val[labels_val[i].item()] +=1

In [395]:
inv_lams_val= (1- lams_val/20)*0.75
inv_lams_val = (lams_val<11) *0.75
inv_lams_val

array([0.  , 0.  , 0.75, 0.  , 0.75, 0.75, 0.75, 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.75, 0.  , 0.  , 0.75, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.75, 0.  , 0.75, 0.  , 0.  , 0.75, 0.  , 0.  ,
       0.  , 0.  , 0.75, 0.75, 0.  , 0.  , 0.  , 0.  , 0.75, 0.75, 0.75,
       0.75, 0.  , 0.  , 0.  , 0.75, 0.75, 0.  , 0.75, 0.75, 0.  , 0.75,
       0.75, 0.75, 0.75, 0.  , 0.  , 0.  , 0.75, 0.75, 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.75, 0.75, 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.75, 0.  , 0.75, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.75, 0.75, 0.75, 0.75, 0.  , 0.  , 0.  , 0.  , 0.75, 0.  ,
       0.75, 0.75, 0.  , 0.  , 0.  , 0.75, 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.75, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.

In [406]:
mixed_clf = pickle.load(open("results_sklearn/config_convex_comb_1/13-Sep-2022-00-50-56/final_model.sav", 'rb'))
i = 1
mixed_clf.coef_ = i*t_norm + (1-i)*i_norm
mixed_clf.intercept_ = i*text_only_balanced_clf.intercept_/text_only_balanced_clf.intercept_.max() 
+ (1-i)*image_only_clf_oversampled.intercept_/image_only_clf_oversampled.intercept_.max()
preds = mixed_clf.predict(image_embeddings)
temp_acc = all_accs_fg(preds, labels.numpy(), fg_indices)
all_accs_val.append(temp_acc)

['0.45370', '0.51200', '0.46115', '0.38591', '0.47511', '0.41215', '0.44190', '0.41109', '0.43438', '0.47784', '0.50174']


In [405]:
mixed_clf = pickle.load(open("results_sklearn/config_convex_comb_1/13-Sep-2022-00-50-56/final_model.sav", 'rb'))
i = 1
mixed_clf.coef_ = i*text_only_balanced_clf.coef_ + (1-i)*image_only_clf_oversampled.coef_
mixed_clf.intercept_ = i*text_only_balanced_clf.intercept_
+ (1-i)*image_only_clf_oversampled.intercept_
preds = mixed_clf.predict(image_embeddings)
temp_acc = all_accs_fg(preds, labels.numpy(), fg_indices)
all_accs_val.append(temp_acc)

['0.54418', '0.33600', '0.50962', '0.50727', '0.54089', '0.53620', '0.56143', '0.54078', '0.53225', '0.55186', '0.56743']


In [398]:
t_norm = text_only_balanced_clf.coef_/text_only_balanced_clf.coef_.max(axis=0)
i_norm = image_only_clf_oversampled.coef_/image_only_clf_oversampled.coef_.max(axis=0)

In [383]:
mixed_clf.coef_

492.3570421989682

In [384]:
np.linalg.norm(image_only_clf_oversampled.coef_)

1073.2520928872243

In [385]:
np.linalg.norm(t_norm)

94.34817983660311

In [386]:
np.linalg.norm(i_norm)

108.45128097811237

In [380]:
text_only_balanced_clf.coef_.shape

(1000, 1024)

In [24]:
image_only_clf_oversampled = pickle.load(open("results_sklearn/config_image_linear_probe_0/c_10/13-Nov-2022-20-07-26/final_model.sav", 'rb'))

In [25]:
import torchvision.transforms as T
from torchvision.transforms import InterpolationMode

BICUBIC = InterpolationMode.BICUBIC
rc = T.RandomResizedCrop(size=224, scale=(0.8, 1), interpolation=BICUBIC)
rg = T.RandomGrayscale()
cj = T.ColorJitter(brightness=0.6, contrast=0.6, saturation=0.6, hue=0)
# t4 = T.GaussianBlur(kernel_size=(3, 5), sigma=(0.1, 2))
gb = T.GaussianBlur(kernel_size=(5, 5), sigma=(2))
# t4 = T.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2))

In [29]:
from scipy import spatial

l = 2

cropped_ims = []
cropped_im_labels = []
cropped_im_ind_labels = []
mixed_clf = pickle.load(open("results_sklearn/config_convex_comb_1/13-Sep-2022-00-50-56/final_model.sav", 'rb'))
from scipy import stats

total_acc = np.zeros(1000)
for im, label, _, path in tqdm(dataset_test):
    for k in range(len(im)):
        cropped_ims = []
        cropped_im_labels = []
        if label[k].item() == l:
            print(f"Found {l} - freq {freqs[l]}")
            curr_im = im[k]
            cropped_ims.append(curr_im)
            cropped_im_labels.append(label[k].item())
#             cropped_ims.append(t1(curr_im))
#             cropped_im_labels.append(label[k].item())
            cropped_ims.append(rg(curr_im))
            cropped_im_labels.append(label[k].item())
#             cropped_ims.append(t2(curr_im))
#             cropped_im_labels.append(label[k].item())
            cropped_ims.append(cj(curr_im))
            cropped_im_labels.append(label[k].item())
#             cropped_ims.append(t4(curr_im))
#             cropped_im_labels.append(label[k].item())
            cropped_ims.append(gb(curr_im))
            cropped_im_labels.append(label[k].item())
            cropped_ims = torch.stack(cropped_ims)
            with torch.no_grad():
                x = visual_model(cropped_ims.half()).float()
                x = x / x.norm(dim=-1, keepdim=True)
            final_pred = []
            best_agreement = 0
            default_pred = 0
            all_coef_preds = []
            all_coef_pred_probas = []
            all_agreements = []
            best_mixer = 0
            for ii in range(0, 21):
                i = ii/20.0
                mixed_clf.coef_ = i*text_only_balanced_clf.coef_ + (1-i)*image_only_clf_oversampled.coef_
                mixed_clf.intercept_ = i*text_only_balanced_clf.intercept_ + (1-i)*image_only_clf_oversampled.intercept_
                probas = mixed_clf.predict_proba(x.cpu().detach().numpy())
                preds = mixed_clf.predict(x.cpu().detach().numpy())
                all_coef_preds.append(preds)
                all_coef_pred_probas.append(np.var(probas))
                agreement = 0
                if i == 0.65:
                    default_pred = preds[0]
                for n in range(1,len(cropped_im_labels)):
                    agreement+= 1 - spatial.distance.cosine(probas[0], probas[n])
                agreement /= (len(cropped_im_labels)-1)
                all_agreements.append(agreement)
                if agreement > best_agreement and ii >0 and ii < 20:
                    final_pred = [ii]
                    best_agreement = agreement
                    best_mixer = i
            final_pred = all_coef_preds[final_pred[0]][0]
            print(default_pred, final_pred, best_agreement, best_mixer)
            print(CLASSES[default_pred], CLASSES[final_pred])
            if final_pred == label[k].item():
                total_acc[label[k].item()] +=1
                if default_pred != label[k].item():
                    print("*********")
                    print(all_coef_preds)
                    print(all_agreements)
            elif default_pred == label[k].item():
                if final_pred != label[k].item():
                    print("xxxxxxxx")
                    print(all_coef_preds)
                    print(all_agreements)
#             if default_pred != label[k].item():
#                 print("xxxxxxxx")
#                 print(all_coef_preds)
#                 print(np.round(all_coef_pred_probas,6))
                

print(cwa[l], total_acc[l]*2)

  0%|          | 0/50 [00:00<?, ?it/s]

Found 2 - freq 516
2 2 0.9502709503365389 0.05
great white shark great white shark
Found 2 - freq 516
2 2 0.8856829849710576 0.95
great white shark great white shark
Found 2 - freq 516
921 921 0.9423902656332848 0.95
dust jacket dust jacket
Found 2 - freq 516
2 2 0.86227508554837 0.95
great white shark great white shark
Found 2 - freq 516
2 2 0.7802713484927523 0.95
great white shark great white shark
Found 2 - freq 516
148 148 0.9289819907710446 0.95
killer whale killer whale
Found 2 - freq 516
3 3 0.9271258767774206 0.95
tiger shark tiger shark
Found 2 - freq 516
3 3 0.8477640238278562 0.95
tiger shark tiger shark
Found 2 - freq 516
3 3 0.9520554633667632 0.7
tiger shark tiger shark
Found 2 - freq 516
2 3 0.9049691776641001 0.95
great white shark tiger shark
xxxxxxxx
[array([2, 2, 4, 4]), array([2, 2, 4, 4]), array([2, 2, 4, 4]), array([2, 2, 4, 4]), array([2, 2, 4, 4]), array([2, 2, 4, 4]), array([2, 2, 4, 4]), array([2, 2, 4, 4]), array([2, 2, 4, 4]), array([2, 2, 4, 4]), array([2,

NameError: name 'cwa' is not defined

In [27]:
# Find augmentations for which entropy of text and image classifiers are similar
import importlib
from data_loader import dataloaders as dataloader
importlib.reload(dataloader)

<module 'data_loader.dataloaders' from '/coc/testnvme/bdevnani3/long_tail_lang/data_loader/dataloaders.py'>

In [28]:
# Image Embeddings
og_proba = image_only_clf_oversampled.predict_proba(image_embeddings)
og_preds = image_only_clf_oversampled.predict(image_embeddings)
all_accs_p(og_preds)

og_text_proba = text_only_balanced_clf.predict_proba(image_embeddings)
og_text_preds = text_only_balanced_clf.predict(image_embeddings)
all_accs_p(og_text_preds)

0.624/0.724/0.608/0.388
0.544/0.558/0.542/0.512


[0.54418, 0.5579539641943734, 0.542029598308668, 0.5120588235294118]

In [29]:
def pw_cosine_sim(a,b):
    """a and b are the same dimensions"""
    
    sim = 0
    for i in range(a.shape[0]):
        sim+= 1 - spatial.distance.cosine(a[i], b[i])
        
    sim /= a.shape[0]
    return sim

In [30]:
def vis_enc(im):
    x = visual_model(im.half()).float()
    x = x / x.norm(dim=-1, keepdim=True)
    return x

In [31]:
d = dataloader.load_data(
    data_root="./datasets/ImageNet/",
    dataset="ImageNet_LT",
    phase="test",
    batch_size=1000,
    sampler_dic=None,
    num_workers=10,
    type="LT_Dataset_transform_mix",
    prompt_set="Imagenet",
#     override_transform = [T.RandomGrayscale(p=0.5)]
)

dataset_test = d[0]

all_og = []
all_cj = []
all_hf = []
all_rc = []
all_gb = []
all_gs = []
ls = []
for im_cj, im_hf, im_rc, im_gb, im_gs, im_og, label, _, _ in tqdm(dataset_test):
    with torch.no_grad():
        all_cj.append(vis_enc(im_cj))
        all_hf.append(vis_enc(im_hf))
        all_rc.append(vis_enc(im_rc))
        all_gb.append(vis_enc(im_gb))
        all_gs.append(vis_enc(im_gs))
        all_og.append(vis_enc(im_og))
        ls.append(label)
        
all_cj = torch.cat(all_cj,0).cpu().numpy()
all_hf = torch.cat(all_hf,0).cpu().numpy()
all_rc = torch.cat(all_rc,0).cpu().numpy()
all_gb = torch.cat(all_gb,0).cpu().numpy()
all_gs = torch.cat(all_gs,0).cpu().numpy()
all_og = torch.cat(all_og,0).cpu().numpy()
ls = torch.cat(ls,0).cpu().numpy()

print(all_cj.shape, ls.shape)

Loading data from /nethome/bdevnani3/flash1/long_tail_lang/data/ImageNet_LT/ImageNet_LT_test.txt
Use data transformation: Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=None)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
)
***********************DATASET: test LT_Dataset_transform_mix
test 50000
No sampler.
Shuffle is True.


  0%|          | 0/50 [00:00<?, ?it/s]

(50000, 1024) (50000,)


In [70]:
preds = image_only_clf_oversampled.predict(all_cj)
print(np.sum(preds == ls)/500)

preds = image_only_clf_oversampled.predict(all_hf)
print(np.sum(preds == ls)/500)

preds = image_only_clf_oversampled.predict(all_rc)
print(np.sum(preds == ls)/500)

preds = image_only_clf_oversampled.predict(all_gb)
print(np.sum(preds == ls)/500)

preds = image_only_clf_oversampled.predict(all_gs)
print(np.sum(preds == ls)/500)

preds = image_only_clf_oversampled.predict(all_og)
print(np.sum(preds == ls)/500)

60.968
61.612
61.634
61.632
61.674
62.326


In [71]:
preds = text_only_balanced_clf.predict(all_cj)
print(np.sum(preds == ls)/500)

preds = text_only_balanced_clf.predict(all_hf)
print(np.sum(preds == ls)/500)

preds = text_only_balanced_clf.predict(all_rc)
print(np.sum(preds == ls)/500)

preds = text_only_balanced_clf.predict(all_gb)
print(np.sum(preds == ls)/500)

preds = text_only_balanced_clf.predict(all_gs)
print(np.sum(preds == ls)/500)

preds = text_only_balanced_clf.predict(all_og)
print(np.sum(preds == ls)/500)

52.626
53.422
53.572
53.33
53.37
54.39


In [72]:
from scipy import spatial

l = 3

mixed_clf = pickle.load(open("results_sklearn/config_convex_comb_1/13-Sep-2022-00-50-56/final_model.sav", 'rb'))
from scipy import stats

total_acc = np.zeros(1000)
best_acc = np.zeros(1000)
def_acc = np.zeros(1000)
for l in tqdm(range(1000)):
    for z in range(50000):
        if ls[z] == l:
            final_pred = []
            best_agreement = 0
            default_pred = 0
            all_coef_preds = []
            all_coef_pred_probas = []
            all_agreements = []
            best_mixer = 0
            x = np.vstack([all_og[z], all_hf[z], all_rc[z], all_gb[z], all_gs[z]])
            best_acc_temp = 0
            for ii in range(0, 21):
                i = ii/20.0
                mixed_clf.coef_ = i*conv_comb_0_8_clf.coef_ + (1-i)*image_only_clf_oversampled.coef_
                mixed_clf.intercept_ = i*conv_comb_0_8_clf.intercept_ + (1-i)*image_only_clf_oversampled.intercept_
                probas = mixed_clf.predict_proba(x)
                preds = mixed_clf.predict(x)
                all_coef_preds.append(preds)
                all_coef_pred_probas.append(probas)
                agreement = 0
                if i == 0.65:
                    default_pred = preds[0]
                for n in range(1,probas.shape[0]):
                    agreement+= 1 - spatial.distance.cosine(probas[0], probas[n])
                agreement /= (probas.shape[0]-1)
                all_agreements.append(agreement)
                if agreement > best_agreement and ii < 20:
                    final_pred = [ii]
                    best_agreement = agreement
                    best_mixer = i
                if preds[0] == ls[z]:
                    best_acc_temp = 1
            final_pred = all_coef_preds[final_pred[0]][0]
#             print(default_pred, final_pred, best_agreement, best_mixer)
#             print(CLASSES[default_pred], CLASSES[final_pred])
            best_acc[ls[z]] += best_acc_temp

            if final_pred == ls[z]:
                total_acc[ls[z]] +=1
#                 if default_pred != ls[z]:
#                     print("*********")
#                     print(all_coef_preds)
#                     print(all_agreements)

            if default_pred == ls[z]:
                def_acc[ls[z]] +=1
#                 if final_pred != ls[z]:
#                     print("xxxxxxxx")
#                     print(all_coef_preds)
#                     print(all_agreements)


    print(l, best_acc[l]*2, def_acc[l]*2, total_acc[l]*2)
print(np.mean(best_acc), np.mean(def_acc), np.mean(total_acc))

  0%|          | 0/1000 [00:00<?, ?it/s]

0 94.0 92.0 94.0
1 94.0 92.0 92.0
2 72.0 70.0 66.0
3 70.0 60.0 68.0
4 92.0 90.0 86.0
5 70.0 64.0 62.0
6 56.0 42.0 38.0
7 72.0 72.0 70.0
8 86.0 72.0 82.0
9 94.0 94.0 92.0
10 86.0 86.0 86.0
11 90.0 88.0 88.0
12 80.0 80.0 76.0
13 94.0 92.0 92.0
14 92.0 82.0 88.0
15 74.0 72.0 68.0
16 86.0 86.0 86.0
17 82.0 76.0 68.0
18 88.0 86.0 88.0
19 96.0 96.0 94.0
20 84.0 82.0 82.0
21 86.0 80.0 80.0
22 88.0 86.0 86.0
23 84.0 78.0 80.0
24 96.0 96.0 96.0
25 86.0 84.0 84.0
26 62.0 58.0 60.0
27 74.0 70.0 72.0
28 82.0 80.0 80.0
29 84.0 80.0 82.0
30 84.0 82.0 82.0
31 82.0 82.0 82.0
32 36.0 32.0 34.0
33 62.0 58.0 60.0
34 66.0 64.0 62.0
35 60.0 48.0 50.0
36 44.0 38.0 40.0
37 74.0 70.0 66.0
38 56.0 50.0 50.0
39 78.0 70.0 66.0
40 64.0 56.0 48.0
41 28.0 20.0 18.0
42 68.0 64.0 60.0
43 72.0 62.0 70.0
44 66.0 56.0 58.0
45 70.0 68.0 68.0
46 60.0 32.0 48.0
47 68.0 60.0 56.0
48 88.0 88.0 82.0
49 82.0 74.0 72.0
50 64.0 52.0 54.0
51 94.0 90.0 88.0
52 66.0 40.0 54.0
53 82.0 78.0 78.0
54 20.0 8.0 16.0
55 60.0 48.0 46.0
56 

438 40.0 34.0 34.0
439 98.0 92.0 98.0
440 72.0 70.0 64.0
441 86.0 80.0 86.0
442 64.0 60.0 58.0
443 64.0 60.0 60.0
444 96.0 94.0 94.0
445 58.0 46.0 52.0
446 44.0 40.0 42.0
447 58.0 50.0 56.0
448 82.0 76.0 72.0
449 94.0 92.0 92.0
450 84.0 82.0 82.0
451 78.0 72.0 74.0
452 68.0 58.0 66.0
453 68.0 68.0 68.0
454 92.0 92.0 92.0
455 50.0 46.0 44.0
456 72.0 60.0 66.0
457 54.0 44.0 46.0
458 96.0 96.0 94.0
459 54.0 50.0 48.0
460 72.0 66.0 64.0
461 26.0 18.0 14.0
462 64.0 60.0 52.0
463 24.0 22.0 18.0
464 52.0 44.0 46.0
465 70.0 66.0 68.0
466 100.0 100.0 96.0
467 80.0 78.0 74.0
468 76.0 70.0 76.0
469 38.0 30.0 34.0
470 70.0 64.0 66.0
471 86.0 84.0 82.0
472 92.0 82.0 88.0
473 60.0 48.0 52.0
474 74.0 70.0 66.0
475 92.0 92.0 92.0
476 98.0 98.0 98.0
477 84.0 80.0 76.0
478 58.0 56.0 50.0
479 52.0 44.0 46.0
480 86.0 84.0 86.0
481 66.0 62.0 62.0
482 14.0 10.0 6.0
483 76.0 76.0 76.0
484 70.0 66.0 70.0
485 70.0 62.0 64.0
486 78.0 74.0 76.0
487 74.0 56.0 68.0
488 30.0 28.0 28.0
489 54.0 40.0 38.0
490 66.0 54

870 76.0 64.0 68.0
871 60.0 58.0 58.0
872 74.0 64.0 62.0
873 82.0 82.0 78.0
874 94.0 94.0 92.0
875 64.0 52.0 60.0
876 68.0 48.0 50.0
877 54.0 50.0 50.0
878 86.0 72.0 66.0
879 72.0 64.0 70.0
880 84.0 74.0 74.0
881 68.0 66.0 64.0
882 80.0 72.0 74.0
883 34.0 32.0 34.0
884 82.0 76.0 78.0
885 28.0 20.0 22.0
886 92.0 86.0 90.0
887 20.0 16.0 12.0
888 70.0 68.0 64.0
889 54.0 50.0 52.0
890 94.0 92.0 94.0
891 64.0 60.0 58.0
892 80.0 78.0 74.0
893 66.0 54.0 58.0
894 56.0 46.0 44.0
895 92.0 92.0 92.0
896 64.0 60.0 60.0
897 78.0 76.0 76.0
898 70.0 66.0 68.0
899 28.0 16.0 28.0
900 84.0 84.0 82.0
901 76.0 74.0 72.0
902 60.0 50.0 56.0
903 90.0 86.0 82.0
904 60.0 52.0 56.0
905 52.0 48.0 50.0
906 40.0 40.0 38.0
907 72.0 68.0 66.0
908 58.0 52.0 50.0
909 44.0 34.0 32.0
910 48.0 40.0 46.0
911 30.0 28.0 26.0
912 56.0 54.0 54.0
913 84.0 82.0 84.0
914 66.0 66.0 66.0
915 90.0 86.0 88.0
916 84.0 84.0 84.0
917 76.0 72.0 70.0
918 88.0 86.0 84.0
919 80.0 80.0 78.0
920 82.0 80.0 80.0
921 84.0 70.0 76.0
922 98.0 96.

In [73]:
from scipy import spatial

l = 3

mixed_clf = pickle.load(open("results_sklearn/config_convex_comb_1/13-Sep-2022-00-50-56/final_model.sav", 'rb'))
from scipy import stats

total_acc = np.zeros(1000)
best_acc = np.zeros(1000)
def_acc = np.zeros(1000)
for l in tqdm(range(1000)):
    for z in range(50000):
        if ls[z] == l:
            final_pred = []
            best_agreement = 0
            default_pred = 0
            all_coef_preds = []
            all_coef_pred_probas = []
            all_agreements = []
            best_mixer = 0
            x = np.vstack([all_og[z], all_hf[z], all_rc[z], all_gb[z], all_gs[z]])
            best_acc_temp = 0
            for ii in range(0, 21):
                i = ii/20.0
                mixed_clf.coef_ = i*text_only_balanced_clf.coef_ + (1-i)*image_only_clf_oversampled.coef_
                mixed_clf.intercept_ = i*text_only_balanced_clf.intercept_ + (1-i)*image_only_clf_oversampled.intercept_
                probas = mixed_clf.predict_proba(x)
                preds = mixed_clf.predict(x)
                all_coef_preds.append(preds)
                all_coef_pred_probas.append(probas)
                agreement = 0
                if i == 0.65:
                    default_pred = preds[0]
                for n in range(1,probas.shape[0]):
                    agreement+= 1 - spatial.distance.cosine(probas[0], probas[n])
                agreement /= (probas.shape[0]-1)
                all_agreements.append(agreement)
                if agreement > best_agreement and ii < 20:
                    final_pred = [ii]
                    best_agreement = agreement
                    best_mixer = i
                if preds[0] == ls[z]:
                    best_acc_temp = 1
            final_pred = all_coef_preds[final_pred[0]][0]
#             print(default_pred, final_pred, best_agreement, best_mixer)
#             print(CLASSES[default_pred], CLASSES[final_pred])
            best_acc[ls[z]] += best_acc_temp

            if final_pred == ls[z]:
                total_acc[ls[z]] +=1
#                 if default_pred != ls[z]:
#                     print("*********")
#                     print(all_coef_preds)
#                     print(all_agreements)

            if default_pred == ls[z]:
                def_acc[ls[z]] +=1
#                 if final_pred != ls[z]:
#                     print("xxxxxxxx")
#                     print(all_coef_preds)
#                     print(all_agreements)


    print(l, best_acc[l]*2, def_acc[l]*2, total_acc[l]*2)
print(np.mean(best_acc), np.mean(def_acc), np.mean(total_acc))

  0%|          | 0/1000 [00:00<?, ?it/s]

0 96.0 96.0 96.0
1 92.0 90.0 86.0
2 74.0 72.0 62.0
3 72.0 64.0 56.0
4 90.0 82.0 78.0
5 66.0 62.0 62.0
6 70.0 62.0 56.0
7 64.0 48.0 44.0
8 98.0 94.0 92.0
9 94.0 92.0 86.0
10 92.0 90.0 90.0
11 94.0 88.0 88.0
12 82.0 76.0 62.0
13 98.0 92.0 94.0
14 96.0 92.0 90.0
15 84.0 70.0 82.0
16 92.0 92.0 92.0
17 88.0 76.0 86.0
18 90.0 84.0 86.0
19 94.0 90.0 78.0
20 90.0 86.0 88.0
21 86.0 86.0 84.0
22 88.0 82.0 74.0
23 86.0 82.0 74.0
24 96.0 96.0 96.0
25 86.0 82.0 82.0
26 72.0 58.0 54.0
27 76.0 74.0 74.0
28 84.0 80.0 76.0
29 86.0 82.0 72.0
30 86.0 84.0 86.0
31 82.0 82.0 66.0
32 38.0 34.0 28.0
33 72.0 68.0 64.0
34 68.0 64.0 60.0
35 64.0 46.0 42.0
36 66.0 48.0 52.0
37 72.0 66.0 60.0
38 68.0 50.0 64.0
39 82.0 74.0 66.0
40 68.0 54.0 50.0
41 66.0 28.0 46.0
42 82.0 68.0 72.0
43 72.0 58.0 38.0
44 64.0 58.0 48.0
45 76.0 68.0 68.0
46 64.0 32.0 20.0
47 68.0 56.0 54.0
48 92.0 90.0 86.0
49 62.0 54.0 52.0
50 74.0 64.0 58.0
51 94.0 86.0 90.0
52 72.0 56.0 40.0
53 76.0 74.0 64.0
54 30.0 8.0 6.0
55 60.0 42.0 20.0
56 6

439 98.0 92.0 86.0
440 82.0 66.0 70.0
441 88.0 84.0 80.0
442 66.0 50.0 50.0
443 78.0 64.0 72.0
444 98.0 94.0 96.0
445 58.0 44.0 28.0
446 52.0 38.0 34.0
447 68.0 56.0 52.0
448 74.0 70.0 72.0
449 96.0 92.0 86.0
450 88.0 80.0 86.0
451 82.0 70.0 66.0
452 72.0 60.0 62.0
453 72.0 68.0 66.0
454 92.0 92.0 90.0
455 48.0 40.0 34.0
456 78.0 62.0 48.0
457 60.0 52.0 44.0
458 98.0 94.0 96.0
459 52.0 44.0 48.0
460 76.0 66.0 56.0
461 26.0 14.0 8.0
462 68.0 62.0 64.0
463 46.0 32.0 36.0
464 56.0 50.0 44.0
465 78.0 72.0 76.0
466 100.0 100.0 100.0
467 78.0 78.0 72.0
468 80.0 64.0 68.0
469 42.0 30.0 24.0
470 78.0 70.0 64.0
471 92.0 88.0 90.0
472 92.0 84.0 76.0
473 66.0 50.0 40.0
474 90.0 72.0 70.0
475 96.0 92.0 96.0
476 100.0 100.0 98.0
477 90.0 82.0 84.0
478 62.0 52.0 48.0
479 52.0 44.0 44.0
480 90.0 90.0 82.0
481 74.0 64.0 68.0
482 12.0 4.0 6.0
483 76.0 76.0 74.0
484 82.0 64.0 62.0
485 70.0 66.0 48.0
486 96.0 84.0 88.0
487 80.0 78.0 74.0
488 30.0 24.0 22.0
489 50.0 38.0 34.0
490 62.0 58.0 56.0
491 52.0 3

871 78.0 54.0 62.0
872 80.0 70.0 72.0
873 86.0 82.0 84.0
874 94.0 92.0 90.0
875 76.0 62.0 52.0
876 52.0 22.0 20.0
877 74.0 54.0 60.0
878 92.0 86.0 70.0
879 74.0 70.0 62.0
880 82.0 74.0 70.0
881 76.0 64.0 72.0
882 82.0 78.0 72.0
883 66.0 46.0 56.0
884 84.0 78.0 76.0
885 24.0 16.0 18.0
886 94.0 90.0 90.0
887 38.0 20.0 24.0
888 90.0 74.0 76.0
889 60.0 52.0 38.0
890 100.0 92.0 94.0
891 64.0 60.0 50.0
892 84.0 80.0 80.0
893 68.0 54.0 60.0
894 66.0 58.0 56.0
895 94.0 94.0 94.0
896 72.0 60.0 54.0
897 76.0 76.0 68.0
898 70.0 58.0 50.0
899 32.0 12.0 6.0
900 88.0 84.0 86.0
901 80.0 70.0 68.0
902 68.0 42.0 24.0
903 92.0 86.0 82.0
904 58.0 50.0 46.0
905 52.0 42.0 30.0
906 56.0 44.0 46.0
907 78.0 74.0 72.0
908 52.0 40.0 38.0
909 54.0 46.0 42.0
910 58.0 48.0 40.0
911 42.0 36.0 32.0
912 64.0 54.0 54.0
913 82.0 76.0 72.0
914 78.0 66.0 58.0
915 94.0 88.0 84.0
916 96.0 90.0 96.0
917 94.0 82.0 84.0
918 86.0 86.0 80.0
919 86.0 80.0 80.0
920 88.0 82.0 76.0
921 84.0 78.0 76.0
922 100.0 96.0 94.0
923 64.0 48

In [56]:
conv_comb_0_8_clf = pickle.load(open("results_sklearn/config_convex_comb_1/class_balanced/13-Sep-2022-13-41-17/final_model.sav", 'rb'))


In [ ]:
from scipy import spatial

l = 3

mixed_clf = pickle.load(open("results_sklearn/config_convex_comb_1/13-Sep-2022-00-50-56/final_model.sav", 'rb'))
from scipy import stats

total_acc = np.zeros(1000)
best_acc = np.zeros(1000)
def_acc = np.zeros(1000)
for z in tqdm(range(50000)):
    final_pred = []
    best_agreement = 0
    default_pred = 0
    all_coef_preds = []
    all_coef_pred_probas = []
    all_agreements = []
    best_mixer = 0
    x = np.vstack([all_og[z], all_hf[z], all_rc[z], all_gb[z], all_gs[z]])
    best_acc_temp = 0
    for ii in range(0, 21):
        i = ii/20.0
        mixed_clf.coef_ = i*conv_comb_0_8_clf.coef_ + (1-i)*image_only_clf_oversampled.coef_
        mixed_clf.intercept_ = i*conv_comb_0_8_clf.intercept_ + (1-i)*image_only_clf_oversampled.intercept_
        probas = mixed_clf.predict_proba(x)
        preds = mixed_clf.predict(x)
        all_coef_preds.append(preds)
        all_coef_pred_probas.append(probas)
        agreement = 0
        if i == 0.65:
            default_pred = preds[0]
        agreement+= 1 - spatial.distance.cosine(probas[1], probas[2])
        all_agreements.append(agreement)
        if agreement > best_agreement and ii < 20 and ii>0:
            final_pred = [ii]
            best_agreement = agreement
            best_mixer = i
        if preds[0] == ls[z]:
            best_acc_temp = 1
    final_pred = all_coef_preds[final_pred[0]][0]
    best_acc[ls[z]] += best_acc_temp

    if final_pred == ls[z]:
        total_acc[ls[z]] +=1

    if default_pred == ls[z]:
        def_acc[ls[z]] +=1

    if z% 500 == 0:
        print(np.sum(best_acc), np.sum(def_acc), np.sum(total_acc))
#     print(l, best_acc[l]*2, def_acc[l]*2, total_acc[l]*2)
print(np.mean(best_acc), np.mean(def_acc), np.mean(total_acc))

  0%|          | 0/50000 [00:00<?, ?it/s]

1.0 1.0 1.0
365.0 351.0 349.0
